# HowTo-2-CalculatingForcedPhotometry

In [1]:
from qso_toolbox import catalog_tools as ct, image_tools as it, photometry_tools as pt, utils as ut

# 0) Loading the test file

### We begin again my loading our Stripe 82 milliquas test file into memory.

In [2]:
import pandas as pd 
from astropy.io import fits 
from astropy.table import Table

df_hdf = pd.read_hdf('./data/stripe82_milliquas_190210.hdf5',key='data')

# We use the first 100 entries of the test set we used for HowTo-1-DownloadingImageCutouts
test_set = df_hdf.query('340 < mq_ra < 350 and -1.26 < mq_dec < 0')[:200]

# 1) Calculating forced photometry

### There is a lot that goes into the calculation of the forced photometry and it is hidden in a range of routines in image_tools.py. 

## The main routine to calculate forced photometry is:

def get_forced_photometry(table, ra_col_name, dec_col_name, surveys,
                          bands, apertures, fovs, image_folder_path,
                          auto_download=True,
                          verbosity=0):
    """Calculate forced photometry for all objects in the table Data Frame.

    In the current version of this routine forced photometry calculations for
    the following surveys and bands is available:
    survey: 'desdr1'
        bands: 'grizy'
    survey: "unwise-allwise, unwise-neo1, unwise-neo2, "unwise-neo3"
        bands: 'w1w2w3s4

    This function takes a table object (astropy table, astropy fitstable or
    DataFrame) with specified Ra and Dec. It eiher looks for the image
    cutouts associated with each survey/band/fov entry or automatically
    downloads them, if specified. If the image cutouts are found forced
    photometry is calculated within the specified aperture.

    For each survey/band the following columns are added to the input table:
    forced_[survey]_mag_[band]
        Forced photometry magnitude for the object in the given survey/band.
        The magnitudes are all in the AB system
    forced_[survey]_flux_[band]
        Forced photometry flux for the object in the given survey/band
    forced_[survey]_sn_[band]
        Forced photometry S/N for the object in the given survey/band
    forced_[survey]_magerr_[band]
        Forced photometry magnitude error for the object in the given
        survey/band
    forced_[survey]_comment_[band]
        A comment with regard to the forced photometry calculation for each
        object in the given survey/band.
        If the forced photometry calculation is successful the comment will
        give the used apertures: 'ap_[aperture in arcseconds]'
        If the forced photometry calculation is unsuccessfull the comment will
        reflect the problem:
        'image_too_small': cutout image is too small to calculate the forced
         photometry (minimum pixel size 50)
        'image_not_available': cutout image could not be found and/or downloaded
        'crashed': bad things happened! (Check try-except clause in
        calculate_forced_aperture_photometry)

    Lists of equal length need to be supplied to surveys, bands, apertures and
    fovs.

    :param table: table object
        Input data table with at least RA and Decl. columns
    :param ra_col_name: string
        Exact string for the RA column in the table
    :param dec_col_name: string
        Exact string for the Decl. column in the table
    :param surveys: list of strings
        List of survey names, length has to be equal to bands, apertures and
        fovs
    :param bands: list of strings
        List of band names, length has to be equal to surveys, apertures and
        fovs
    :param apertures: list of floats
        List of apertures in arcseconds for forced photometry calculated,
        length has to be equal to surveys, bands and fovs
    :param fovs: list of floats
        Field of view in arcseconds of image cutouts, length has be equal to
        surveys,
        bands and apertures
    :param image_folder_path: string
        Path to the directory where all the images will be stored
    :param auto_download: Boolean
        Switch to enable/disable auto-downloading the cutouts images
    :param verbosity:
        Verbosity > 0 will print verbose statements during the execution
    :return: DataFrame
        Returns a DataFrame with the added columns for the forced photometry
        calculation.
    """

### We start by calculating the forced photometry for DES and unWISE. We turn auto_download to True in order to make sure that we have the cutouts on file. If the files already exist in the specified image_path_folder then they will not be downloaded again.

In [3]:
surveys = ['desdr1','unwise-neo3']
bands = ['z','w1']
apertures = [6, 10]
fovs = [300, 400]

forced_photometry = it.get_forced_photometry(test_set[:10], 'mq_ra', 'mq_dec', surveys, bands, apertures, fovs,
                         './cutouts/', auto_download=True, verbosity=2)

/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


test-1
test-2
test-3
Opened ./cutouts/J224007.14-005332.81_desdr1_z_fov300.fits with a fov of 300 arcseconds
flux:  926737.944563672 sn:  1248.5552219544052
mag:  15.082607636913457
test-2
test-3
Opened ./cutouts/J224007.90-011522.10_desdr1_z_fov300.fits with a fov of 300 arcseconds


/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


flux:  11399.831090738899 sn:  23.208284189748312
mag:  19.857753958699057
test-2
test-3
Opened ./cutouts/J224009.37-003856.60_desdr1_z_fov300.fits with a fov of 300 arcseconds


/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


...

/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

flux:  733.4057422835456 sn:  19.471230647693005
mag:  15.336639235010349
test-2
test-3
Opened ./cutouts/J224017.29-011442.85_unwise-neo3_w1_fov400.fits with a fov of 400 arcseconds
flux:  606.4168450720745 sn:  20.6287553777315
mag:  15.543071858396141
test-2
test-3
Opened ./cutouts/J224018.29-005223.75_unwise-neo3_w1_fov400.fits with a fov of 400 arcseconds
flux:  424.5232174023386 sn:  11.339330973394516
mag:  15.930246382431184
test-2
test-3
Opened ./cutouts/J224018.58-000656.89_unwise-neo3_w1_fov400.fits with a fov of 400 arcseconds


/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

flux:  275.55143321511366 sn:  4.961495060334653
mag:  16.399493314383463
test-2
test-3
Opened ./cutouts/J224022.77-000151.97_unwise-neo3_w1_fov400.fits with a fov of 400 arcseconds
flux:  518.6755886374915 sn:  12.56037668233366
mag:  15.712760478770198


/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

### Let us now take a quick look at the table with the forced photometry information. If you are wondering why some of the forced photometry calculations fail, setting verbosity=2 can be helpful in identifying the problem. 

In [4]:
forced_photometry

,mq_ra,mq_dec,mq_name,mq_z,mq_cite,forced_desdr1_mag_z,forced_desdr1_flux_z,forced_desdr1_sn_z,forced_desdr1_magerr_z,forced_desdr1_z_comment,forced_unwise-neo3_mag_w1,forced_unwise-neo3_flux_w1,forced_unwise-neo3_sn_w1,forced_unwise-neo3_magerr_w1,forced_unwise-neo3_w1_comment
29552,340.029741,-0.892448,PGC 1132088,0.054,PGC,15.082608,926737.944564,1248.555222,0.000870,ap_6,15.307259,9050.998584,273.412687,0.003971,ap_10
29553,340.032899,-1.256140,SDSS J224007.89-011522.1,0.659,DR14Q,19.857754,11399.831091,23.208284,0.046782,ap_6,19.231664,243.744227,8.714552,0.124589,ap_10
29554,340.039043,-0.649055,SDSS J224009.37-003856.5,2.306,DR14Q,20.635514,5569.221234,8.473751,0.128129,ap_6,16.684475,2545.715210,82.019602,0.013238,ap_10
29556,340.047792,-1.246840,SDSS J224011.46-011448.6,1.451,DR14Q,19.140504,22069.803604,38.778825,0.027998,ap_6,18.890900,333.610702,11.731317,0.092550,ap_10
29557,340.055801,-0.901223,SDSS J224013.39-005404.4,0.160,DR14,17.108760,143382.447520,173.016295,0.006275,ap_6,16.946694,1999.503846,64.466602,0.016842,ap_10
29559,340.059501,-0.189209,SDSS J224014.28-001121.1,2.235,DR14,19.041673,24173.011869,32.231579,0.033685,ap_6,18.035639,733.405742,19.471231,0.055761,ap_10
29561,340.072023,-1.245235,SDSS J224017.28-011442.8,0.502,DR14Q,19.227211,20375.848510,35.403200,0.030668,ap_6,18.242072,606.416845,20.628755,0.052632,ap_10
29562,340.076227,-0.873265,SDSS J224018.29-005223.7,2.561,DR14Q,20.962682,4120.285536,5.038796,0.215475,ap_6,18.629246,424.523217,11.339331,0.095750,ap_10
29563,340.077436,-0.115803,SDSS J224018.58-000656.8,2.297,DR14Q,20.446812,6626.362411,9.691486,0.112030,ap_6,19.098493,275.551433,4.961495,0.218832,ap_10
29567,340.094890,-0.031102,SDSS J224022.77-000151.9,0.910,DR14Q,20.322031,7433.403107,11.362643,0.095553,ap_6,18.411760,518.675589,12.560377,0.086441,ap_10


# 2) Calculating forced photometry - Multiprocessing

### The function to calculate forced photometry in multiprocessing mode works analogous to the single core function above. The number of threads spawned is defined by n_jobs and should not exceed the cores on your machine (even if your CPU has multithreading). We have enabled verbose statement (verbosity=2) to get and follow the routine while it is working. 

In [5]:
surveys = ['desdr1','unwise-neo3']
bands = ['z','w1']
apertures = [6, 10]
fovs = [300, 400]

mp_forced_photometry = it.get_forced_photometry_mp(test_set[10:15], 'mq_ra', 'mq_dec', surveys,
                          bands, apertures, fovs, './cutouts/', n_jobs=5,
                          auto_download=True, verbosity=2)

/Users/schindler/anaconda3/lib/python3.7/site-packages/qso_toolbox/image_tools.py:1230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epoch="J")


File ./cutouts//J224023.25-003555.74_desdr1_z*.fits in folder ./cutouts/ not found. Target with RA 340.0968943 and Decl -0.598817File ./cutouts//J224026.75-010057.92_desdr1_z*.fits in folder ./cutouts/ not found. Target with RA 340.1114784 and Decl -1.0160884File ./cutouts//J224028.15-003813.17_desdr1_z*.fits in folder ./cutouts/ not found. Target with RA 340.1172887 and Decl -0.6369927File ./cutouts//J224028.85-010649.87_desdr1_z*.fits in folder ./cutouts/ not found. Target with RA 340.1202098 and Decl -1.1138541000000002



[INFO] SVC: [INFO] SVC: [INFO] SVC: File ./cutouts//J224029.07-011419.77_desdr1_z*.fits in folder ./cutouts/ not found. Target with RA 340.12114110000005 and Decl -1.2388244  [INFO] SVC:  
<pyvo.dal.sia.SIAService object at 0x1254d582d0><pyvo.dal.sia.SIAService object at 0x1254d587d0> <pyvo.dal.sia.SIAService object at 0x1254d582d0>

[INFO] SVC: 
The full image list contains 18 entries
The full image list contains 18 entriesThe full image list contains
 The full i

/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/schindler/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


File ./cutouts//J224026.75-010057.92_unwise-neo3_w1*.fits in folder ./cutouts/ not found. Target with RA 340.1114784 and Decl -1.0160884File ./cutouts//J224023.25-003555.74_unwise-neo3_w1*.fits in folder ./cutouts/ not found. Target with RA 340.0968943 and Decl -0.598817File ./cutouts//J224028.15-003813.17_unwise-neo3_w1*.fits in folder ./cutouts/ not found. Target with RA 340.1172887 and Decl -0.6369927

File ./cutouts//J224028.85-010649.87_unwise-neo3_w1*.fits in folder ./cutouts/ not found. Target with RA 340.1202098 and Decl -1.1138541000000002

File ./cutouts//J224029.07-011419.77_unwise-neo3_w1*.fits in folder ./cutouts/ not found. Target with RA 340.12114110000005 and Decl -1.2388244
Download of J224023.25-003555.74_unwise-neo3_w1_fov400 to ./cutouts/ completed
Download of J224028.85-010649.87_unwise-neo3_w1_fov400 to ./cutouts/ completedDownload of J224029.07-011419.77_unwise-neo3_w1_fov400 to ./cutouts/ completedDownload of J224028.15-003813.17_unwise-neo3_w1_fov400 to ./cutou

### Let us now take a look at the output.

In [6]:
mp_forced_photometry

,mq_ra,mq_dec,mq_name,mq_z,mq_cite,temp_object_name,forced_desdr1_mag_z,forced_desdr1_flux_z,forced_desdr1_sn_z,forced_desdr1_magerr_z,forced_desdr1_z_comment,forced_unwise-neo3_mag_w1,forced_unwise-neo3_flux_w1,forced_unwise-neo3_sn_w1,forced_unwise-neo3_magerr_w1,forced_unwise-neo3_w1_comment
29568,340.096894,-0.598817,FBQS J2240-0035,1.167,DR7Q,J224023.25-003555.74,18.661193,34318.069820,55.446672,0.019582,ap_6,18.238672,608.318667,31.323233,0.034662,ap_10
29570,340.111478,-1.016088,SDSS J224026.75-010057.9,0.845,DR14Q,J224026.75-010057.92,21.088375,3669.865507,5.503431,0.197284,ap_6,19.627609,169.260823,5.448844,0.199260,ap_10
29572,340.117289,-0.636993,SDSS J224028.14-003813.1,0.658,DR14Q,J224028.15-003813.17,18.449057,41723.175230,71.029301,0.015286,ap_6,17.218429,1556.784013,46.482664,0.023358,ap_10
29574,340.120210,-1.113854,2MASX J22402887-0106495,0.127,DR7Q,J224028.85-010649.87,16.251543,315778.705397,528.598813,0.002054,ap_6,-996.301000,NaN,NaN,NaN,masked
29576,340.121141,-1.238824,SDSS J224029.07-011419.7,2.472,DR14Q,J224029.07-011419.77,21.147743,3474.584450,6.093472,0.178180,ap_6,-996.301000,NaN,NaN,NaN,masked
